# Remote driver example

## Connection

Enter connection a valid configuration to demo the remote driver.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
config = {
    "ip": "82.130.102.xxx",
    "user": "karajan",
    "port": 22,
    "key": "$KARAJAN_ACCESS_DIR/id_ed25519",
}

First, import the RemoteUnixDriver.

In [3]:
import os, sys, time, pathlib, pprint
sys.path.append("..")

from datapro.exceptions import *
from datapro.util.file import delete
from datapro.driver.unix import SSHUnixDriver as RemoteUnixDriver

Instantiate the driver.

In [4]:
driver = RemoteUnixDriver(backend=config)

The `connect` function calls the backend's `connect` function and gets original state.

In [5]:
driver.connect()

2019-07-02 13:18:04,322 :: WARNING  :: MainThread :: datapro              ::    unix.py:539 (fan) :: setting fan failed due to no fan path found


In [6]:
pprint.pprint(driver.original_state)

{'fan': None,
 'frequencies': [3902005,
                 3850214,
                 3964806,
                 3735776,
                 3851504,
                 3801054,
                 3956995,
                 3972924],
 'governors': ['performance',
               'performance',
               'performance',
               'performance',
               'performance',
               'performance',
               'performance',
               'performance'],
 'latency': 100}


## Properties

Access to system configurables such as fan, governor, scaling frequencies, and DMA latency, are implemented as properties. Current values can be obtained as if they were regular attributes.

In [7]:
print(driver.frequencies)

[3822531, 3896206, 3927319, 3838743, 3993035, 3921833, 3932225, 3929728]


All settings can be obtained and changed via `setstate` and `getstate`. They both accept/produce a dictionary. `getstate` can accept an "incomplete" dictionary, only the present keys will be used for updating the state.

In [8]:
print(driver.getstate().keys())

2019-07-02 13:18:05,157 :: WARNING  :: MainThread :: datapro              ::    unix.py:539 (fan) :: setting fan failed due to no fan path found


dict_keys(['governors', 'frequencies', 'latency', 'fan'])


The setters are implemented quite defensively and will only allow to set meaningful values, may throw exceptions and produce warnings.

In [9]:
try:
    driver.governors = "non-existent-governor"
except DriverValueError as e:
    print("would raise:", type(e).__name__, "\nwith args:", e)

would raise: Driver->ValueError 
with args: 'non-existent-governor' is not a valid governor (['performance', 'powersave'])


In [10]:
try:
    driver.governors = ["powersave", "powersave"]
except AssertionError as e:
    print("would raise:", type(e).__name__, "\nwith args:", e)

would raise: AssertionError 
with args: governor list and cpu list must match


In [11]:
if "userspace" in driver.original_state["governors"][0]:
    driver.governors = "userspace"
    driver.frequencies = 1900000
    time.sleep(0.1); print(set(driver.governors), driver.frequencies, sep="\n")

The original state can be restored by calling `cleanup` or setting the state to `original_state` manually.

In [12]:
time.sleep(0.1); driver.cleanup()
time.sleep(0.1); pprint.pprint(driver.getstate())

2019-07-02 13:18:07,423 :: WARNING  :: MainThread :: datapro              ::    unix.py:559 (fan) :: setting fan failed due to no fan path found
2019-07-02 13:18:08,358 :: WARNING  :: MainThread :: datapro              ::    unix.py:539 (fan) :: setting fan failed due to no fan path found


{'fan': None,
 'frequencies': [3732026,
                 3802532,
                 3809315,
                 3964198,
                 3706133,
                 3856088,
                 3846491,
                 3955482],
 'governors': ['performance',
               'performance',
               'performance',
               'performance',
               'performance',
               'performance',
               'performance',
               'performance'],
 'latency': 100}


### CPU DMA latency handling

CPU DMA latency handler is implemented properly by opening a file descriptor to the QOS file in devfs, writing a desired value to it (properly formatted as a 32-bit integer in bytes), and keeping the file descriptor opened as long as required.

The handler can be "turned off" by writing a `None` to `driver.latency`.

Latencies set by the machine's power subsystem can be accessed via a property `cpuidle`.

In [13]:
print(driver.cpuidle)

{'POLL': 0, 'C1': 2, 'C1E': 10, 'C3': 70, 'C6': 85, 'C7s': 124, 'C8': 200}


The group id of the latency handler process is saved.

In [14]:
driver.latency = 100
print("Setter pgid:", getattr(driver, "_latency_setter_pgid", None))
print("Current DMA latency:", driver.latency)

Setter pgid: 4184
Current DMA latency: 100


The handler process is deleted with all of its children, and the `_latency_setter_pgid` attribute is removed.

In [15]:
driver.latency = None
print("Setter pgid:", getattr(driver, "_latency_setter_pgid", None))
print("Current DMA latency:", driver.latency)

Setter pgid: None
Current DMA latency: 100


## Transfer and file operations

The driver supports a rudimentary array of file operations:
1. stat, find
2. copy, move, delete, mkdir
3. exists, is_file, is_dir
4. send, fetch

In [16]:
send_command = driver.send(path_from=pathlib.Path("./driver.ipynb"), path_to="")

In [17]:
print("File exists after being sent?", driver.exists("./driver.ipynb"))

File exists after being sent? True


In [18]:
driver.delete("./driver.ipynb")
print("File exists after removal?", driver.exists("./driver.ipynb"))

File exists after removal? False


In [19]:
to = pathlib.Path("from_remote")
if to.exists():
    delete(to)
to.mkdir(parents=True)
found = driver.find(".", query="-type d -name '.ssh'")
print("Found:", found)

Found: None


In [20]:
driver.fetch(path_from=".ssh", path_to=to)
# Verify that files were fetched
print("Exists:", (to / ".ssh").exists())
print("Is dir:", (to / ".ssh").is_dir())
print("Contents:", list(to.rglob("**/*")))
delete(to)

Exists: False
Is dir: False
Contents: [PosixPath('from_remote/authorized_keys')]


## Persistent commands

In addition to the simple command execution, support for running persistent applications was added. This is achieved by running the commands in a bash shell executed with a combination of `nohup` and `disown`. The command outputs and return code are written to unique temporary files, and can be read back from them.

This functionality is used to allow running applications and scripts persistently without resorting to terminal multiplexers.

In [21]:
from datapro.driver._backend import result_to_print_ready_dict

def print_result(result):
    _ = None
    
    if hasattr(result, "persistent"):
        _ = result.persistent
        delattr(result, "persistent")
    
    for k, v in result_to_print_ready_dict(result).items():
        print(f"# {k} {str(''):->{78-len(k)-1}}\n{v}")

    if _:
        k = "persistent"
        print(f"# {k} {str(''):->{78-len(k)-1}}")
        for k, v in _.items():
            print(f"{k+':':<10}{v!r}")
                
        result.persistent = _

In [22]:
sleep_amount=3
pers = driver.persistent("dur={}; sleep $dur && echo \"slept for $dur seconds\"".format(sleep_amount))

If the command has not completed, the value of the `exited` attribute will be `None`. A method `update` is added to
the result which allows updating the aforementioned standard attributes. The following loop shows how one could check for the completion of the command.

The result has the following new helpful attributes:

- `pid` which contains the pid of the `nohup`'ed and `disown`'ed process,
- `persistent` which is a dictionary containing filenames where outputs are stored and the complete command (at key `command`),
- `update` which is a method that can be called to update the values for `exited`, `stdout`, and `stderr`.

In [23]:
print("Waiting for completion...", end="")

while pers.exited is None:
    pers.update()
    print(".", end="", flush=True)
    time.sleep(0.5)
print(" Completed!\n")
assert pers.stdout == "slept for {} seconds".format(sleep_amount)
print_result(pers)

Waiting for completion.......... Completed!

# command ----------------------------------------------------------------------
dur=3; sleep $dur && echo "slept for $dur seconds"
# encoding ---------------------------------------------------------------------
UTF-8
# exited -----------------------------------------------------------------------
0
# stdout -----------------------------------------------------------------------
slept for 3 seconds
# stderr -----------------------------------------------------------------------

# pid --------------------------------------------------------------------------
4399
# persistent -------------------------------------------------------------------
pre:      '/usr/bin/env bash -c'
id:       '2019-07-02_13-18-10_eWHca'
out:      '.nohup-2019-07-02_13-18-10_eWHca-out'
err:      '.nohup-2019-07-02_13-18-10_eWHca-err'
ret:      '.nohup-2019-07-02_13-18-10_eWHca-ret'
rch:      '.nohup-2019-07-02_13-18-10_eWHca-rch'
files:    ['.nohup-2019-07-02_13-18-

### Chaining

In some cases it might be beneficial to run another command once a persistent command finishes (for example, killing applications). This can be accomplished with a `chain` keyword argument to the `persistent` method. The example below also demonstrates the `children` entry in the `persistent` attribute of the return value.

In [24]:
with_chain = driver.persistent("sleep 1 & sleep 2 & sleep 3", chain="echo 'Hello, World!' > some_temp_file.txt")
print("child processes:", with_chain.children)

child processes: [4560, 4561, 4562]


In [25]:
driver.command("while [[ -e /proc/{} ]]; do sleep 0.5; done".format(with_chain.pid))
_ = driver.command("cat some_temp_file.txt && rm some_temp_file.txt")
print("result of chained command:", _.stdout)
with_chain.update()

result of chained command: Hello, World!


## Disconnect

In [26]:
driver.cleanup()

2019-07-02 13:18:19,278 :: WARNING  :: MainThread :: datapro              ::    unix.py:559 (fan) :: setting fan failed due to no fan path found


In [27]:
driver.disconnect()